In [15]:
#https://www.kaggle.com/code/asemsaber/diabetes-detection-using-ml-models/notebook
#https://www.kaggle.com/code/amirazadian/classification-how-to-handle-class-imbalance/notebook  
#https://www.kaggle.com/code/alibabaei78/diabetesprediction
#Imbalanced Data
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import time
import plotly.express as px
from sklearn import metrics
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder , OneHotEncoder , StandardScaler , MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn import svm
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import InstanceHardnessThreshold
lb=LabelEncoder()
MMS=MinMaxScaler()
sc=StandardScaler()
import warnings
warnings.simplefilter(action='ignore')
import pickle

In [16]:
def evaluate_model(model, X, y):
    # Perform cross-validation
    scores = cross_val_score(model, X, y, cv=5)  # 5-fold cross-validation
    # Print cross-validation scores
    print("Cross-Validation Scores:", scores)
    print("Mean CV Accuracy:", scores.mean())

In [17]:
Compare = {"Method":[],"P_Count":[],"N_Count":[],"Score_P":[],"Score_N":[],"Score":[]}
def info_func(df,target,method,ver=""):
    #Scaling the dataset using standard scaler
    DF = df.copy()
    scal_s = preprocessing.StandardScaler()
    Ns = scal_s.fit_transform(DF.drop(columns=target))
    DFs = pd.DataFrame(Ns,columns = list(DF.drop(columns=target)))
    DFs[target] = DF[target]
    #--------------------------------------------------------------------------------------------------------------------------
    # Defining x_train  x_test  y_train  y_test using train_test_split
    ts = 0.15 #test size
    x = DFs.drop(columns=target).values
    y = DFs[target].values.reshape(-1,1)
    x_train, x_test, y_train, y_test=train_test_split(x,y, test_size=ts,random_state=308)
    #-------------------------------------------------------------------------------------------------------------------------------
    #Modeling
    model = RandomForestClassifier(max_depth=5,n_estimators=10,max_features=10, random_state=42)
    model.fit(x_train,y_train)
    evaluate_model(model, df.drop(columns=target), df[target])
    #-------------------------------------------------------------------------------------------------------------------------------
    # Accuracy Metrics
    # Accuracy given whole data to the model
    print("Accuray", metrics.accuracy_score(y,model.predict(x)))
    # Accuracy given test data to the model
    print("Accuracy test: ",metrics.accuracy_score(y_test,model.predict(x_test)))
    # Accuracy given train data to the model
    print("Accuracy train: ",metrics.accuracy_score(y_train,model.predict(x_train)))
    # Classification Report
    print(metrics.classification_report(y,model.predict(x)))
    #-------------------------------------------------------------------------------------------------------------------------------
    # Confusion Matrix
    cm = metrics.confusion_matrix(y,model.predict(x)) #confusion Matrix
    cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = cm)
    fig , ax = plt.subplots(figsize=(10,5))
    cm_display.plot(ax=ax)
    plt.show()
    #-------------------------------------------------------------------------------------------------------------------------------
    # Classification Error
    c = 0
    for i in range(len(DF[target].drop_duplicates())):
        for j in range(len(DF[target].drop_duplicates())):
            if i!=j:
                c += cm[i][j]
    ce = c/float(len(DF.drop(columns=target)))
    print(f"classification Error is : {ce*100}%")
    #-------------------------------------------------------------------------------------------------------------------------------
    dfi = df.groupby(target).size().reset_index(name="Count")
    if method=="None":
        Compare["Method"].append("NoMethod")
    else:
        Compare["Method"].append(method.__class__.__name__+ver)
    Compare["P_Count"].append(dfi[dfi[target]==1].values[0][1])
    Compare["N_Count"].append(dfi[dfi[target]==0].values[0][1])
    Compare["Score_P"].append(metrics.classification_report(y,model.predict(x),output_dict=True)["1"]["f1-score"])
    Compare["Score_N"].append(metrics.classification_report(y,model.predict(x),output_dict=True)["0"]["f1-score"])
    Compare["Score"].append(metrics.accuracy_score(y,model.predict(x)))

In [18]:
def shareplot(df,target):
    dfi = df.groupby(target).size().reset_index(name="Count")
    fig = px.pie(dfi, values='Count', names=target)
    fig.update_traces(textposition='auto', textinfo='percent+label',sort=False,showlegend=False,rotation=90)
    fig.update_layout(title_text='Class label shares',title_x=0.5)
    fig.show()

In [20]:
my_df=pd.read_csv('diabetes_prediction_dataset-Copy.csv')
X=my_df.iloc[:,0:8]
X=X.dropna()
Y=my_df.iloc[:,-1]
my_df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0


In [21]:
my_df.to_pickle('_dataset-Copy.pkl')

In [22]:
X['gender']=lb.fit_transform(X['gender'])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
feature_cols = ['smoking_history']
encoder = OneHotEncoder()
# Perform one-hot encoding and create a DataFrame with the encoded columns
X_encoded = pd.DataFrame(encoder.fit_transform(X[feature_cols].values.reshape(-1, 1)).toarray(),
                         columns=encoder.get_feature_names_out(feature_cols))
# Concatenate the new DataFrame with the original DataFrame
X = pd.concat([X.drop(feature_cols, axis=1), X_encoded], axis=1)

In [ ]:
num_cols = ['age', 'bmi', 'HbA1c_level','blood_glucose_level']
X[num_cols] = sc.fit_transform(X[num_cols])
X[num_cols]

In [ ]:
X=np.array(X)
Y=np.array(Y)

In [ ]:
start_time = time.time()
for i in range(50):    
    B_method = InstanceHardnessThreshold(random_state=42,n_jobs=-1)
    x_IHT , y_IHT = B_method.fit_resample(X,Y)
    X = x_IHT;Y = y_IHT
newdf = pd.DataFrame(x_IHT);newdf["diabetes"]=y_IHT
newdf = newdf.reset_index(drop=True)
end_time = time.time()
# Calculate the elapsed time
elapsed_time = end_time - start_time
print(elapsed_time) 
X=newdf.drop("diabetes", axis=1)
Y=newdf["diabetes"]
X=np.array(X)
Y=np.array(Y)


In [ ]:
x_train , x_test , y_train , y_test = train_test_split(X,Y,train_size=0.80 , random_state=42)

In [ ]:
#Random Forest
rfc=RandomForestClassifier(max_depth=5,n_estimators=10,max_features=10, random_state=42)
rfc.fit(x_train, y_train)
train_score4=rfc.score(x_train,y_train)
print("Random Forest Classifier (test score) : ",train_score4)
accuracy4 = rfc.score(x_test, y_test)
print("Random Forest Classifier (test score) : ", accuracy4)

In [ ]:
# Create a DataFrame with the new data
# new_data = pd.DataFrame({
#     'gender': [0],
#     'age': [80.0],
#     'hypertension': [0],
#     'heart_disease': [1],
#     'bmi': [25.19],
#     'HbA1c_level': [6.6],
#     'blood_glucose_level': [140],
#     'smoking_history_No Info': [0.0],
#     'smoking_history_current': [0.0],
#     'smoking_history_ever': [0.0],
#     'smoking_history_former': [0.0],
#     'smoking_history_never': [1.0],
#     'smoking_history_ not current': [0.0]
#     input_data = (0,80.0,0,1,25.19,6.6,140,0.0,0.0,0.0,0.0,1.0,0.0)
#input_data = (0,20,0,0,27.32,6.6,85,0.0,0.0,0.0,0.0,1.0,0.0) No diabet
#input_data = (0,44,0,0,19.31,6.5,200,0.0,0.0,0.0,0.0,1.0,0.0) Diabet
#input_data = (1,74,0,0,32.2,3.5,200,0.0,0.0,0.0,1.0,0.0,0.0) No diabet
#input_data = (1,52,0,0,29.12,8.8,260,0.0,0.0,0.0,1.0,0.0,0.0) Diabet
#input_data = (0,64,0,0,37.74,7.5,160,0.0,0.0,0.0,0.0,1.0,0.0) Diabet



#input_data = (1,49,0,0,27.32,6.2,240,0.0,0.0,0.0,0.0,1.0,0.0) diabet
#input_data = (1,29,0,0,27.32,5.8,80,1.0,0.0,0.0,0.0,0.0,0.0) No diabet

#(1,58,0,0,27.32,38,80,0.0,0.0,0.0,1.0,0.0,0.0)
#





In [ ]:

# Example input data
input_data = (0,44,0,0,19.31,6.5,200,0.0,0.0,0.0,0.0,1.0,0.0)

# Reshape the input data as we are predicting for one instance
input_data_reshaped = np.array(input_data).reshape(1, -1)

# Select only the numeric columns for scaling
num_cols_indices = [1, 4, 5, 6]  # Indices of numeric columns
input_data_numeric = input_data_reshaped[:, num_cols_indices]

# Scale the numeric columns using the previously fitted StandardScaler instance 'sc'
input_data_scaled = sc.transform(input_data_numeric)

# Replace the numeric columns with the scaled values in the original array
input_data_reshaped[:, num_cols_indices] = input_data_scaled

# Make prediction
prediction = rfc.predict(input_data_reshaped)
print(prediction)


In [ ]:
shareplot(newdf,"diabetes")

In [ ]:
info_func(newdf,"diabetes","None")

In [ ]:
import pickle

In [ ]:
filename='trained_model.sav'
pickle.dump(rfc,open(filename,'wb'))

In [ ]:
loaded_model= pickle.load(open('trained_model.sav','rb'))

In [ ]:
input_data = (1,29,0,0,27.32,5.8,80,1.0,0,0,0,0,0)
# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)
# reshape the array as we are predicting for one instance
input_data_reshaped = (input_data_as_numpy_array.reshape(1,-1))
print(input_data_reshaped)
input_data_df = pd.DataFrame(input_data_reshaped, columns=['gender', 'age', 'smoking_history', 'history_of_diabetes', 'bmi', 'HbA1c_level', 'blood_glucose_level', 'Polyuria', 'Polydipsia', 'sudden_weight_loss', 'weakness', 'Polyphagia', 'Genital_thrush'])
# Select only the numeric columns for scaling
num_cols = ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level']
input_data_df[num_cols] = sc.transform(input_data_df[num_cols])
# Convert the DataFrame back to a NumPy array
input_data_reshaped = np.array(input_data_df)
#prediction = grid_search.predict(input_data_reshaped)
prediction = loaded_model.predict(input_data_reshaped)
print(prediction)
# if (prediction == 0):
#   print('The person is not diabetic')
# else:
#   print('The person is diabetic')
